<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_VGG_16_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision as tv
from torchsummary import summary
import time
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
            torch.cuda.empty_cache()
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [4]:
transoforms = tv.transforms.Compose([tv.transforms.Grayscale(3), tv.transforms.Resize((224, 224)), tv.transforms.ToTensor()])
Batch_size = 64
train_dataset = tv.datasets.EMNIST('.', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('.', split='balanced', train=False, download=True, transform=transoforms)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_size)

In [5]:
train_dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [6]:
print(len(train_dataset.classes))


47


In [7]:
model_vgg16 = tv.models.vgg16(weights=tv.models.VGG16_Weights.IMAGENET1K_V1)

for param in model_vgg16.parameters():
    param.requires_glad = False

classifier_list = list(model_vgg16.classifier.children())
classifier_list.pop()
num_classes = 47
classifier_list.append(nn.Linear(4096,num_classes))
model_vgg16.classifier = nn.Sequential(*classifier_list)

optimizer_vgg16 = torch.optim.Adam(model_vgg16.parameters(), lr=0.001)

In [8]:
summary(model_vgg16.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [9]:
train(model_vgg16, train_data, test_data, optimizer_vgg16,1)

Step 0. time since epoch: 1.323. Train acc: 0.031. Train Loss: 3.898
Step 10. time since epoch: 10.830. Train acc: 0.026. Train Loss: 3.984
Step 20. time since epoch: 20.178. Train acc: 0.024. Train Loss: 3.928
Step 30. time since epoch: 29.548. Train acc: 0.024. Train Loss: 3.909
Step 40. time since epoch: 38.899. Train acc: 0.021. Train Loss: 3.897
Step 50. time since epoch: 48.342. Train acc: 0.020. Train Loss: 3.889
Step 60. time since epoch: 57.808. Train acc: 0.019. Train Loss: 3.884
Step 70. time since epoch: 67.284. Train acc: 0.019. Train Loss: 3.880
Step 80. time since epoch: 76.813. Train acc: 0.019. Train Loss: 3.878
Step 90. time since epoch: 86.287. Train acc: 0.020. Train Loss: 3.875
Step 100. time since epoch: 95.842. Train acc: 0.021. Train Loss: 3.874
Step 110. time since epoch: 105.465. Train acc: 0.021. Train Loss: 3.873
Step 120. time since epoch: 114.961. Train acc: 0.020. Train Loss: 3.871
Step 130. time since epoch: 124.449. Train acc: 0.020. Train Loss: 3.870
S